<a href="https://colab.research.google.com/github/EnX-Ayush/Energy_Consumption/blob/main/Energy_Consumption.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [23]:
!pip install xgboost tensorflow holidays seaborn

In [24]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_absolute_error, mean_squared_error
import xgboost as xgb
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping
import holidays

sns.set(style="whitegrid")
plt.rcParams['figure.figsize'] = [12, 5]


In [25]:
from google.colab import files

df = pd.read_csv("/content/Energy_consumption.csv")
print("Dataset loaded")
print("Columns in dataset:", df.columns.tolist())
df.head()


Dataset loaded
Columns in dataset: ['Timestamp', 'Temperature', 'Humidity', 'SquareFootage', 'Occupancy', 'HVACUsage', 'LightingUsage', 'RenewableEnergy', 'DayOfWeek', 'Holiday', 'EnergyConsumption']


,Timestamp,Temperature,Humidity,SquareFootage,Occupancy,HVACUsage,LightingUsage,RenewableEnergy,DayOfWeek,Holiday,EnergyConsumption
0,2022-01-01 00:00:00,25.139433,43.431581,1565.693999,5,On,Off,2.774699,Monday,No,75.364373
1,2022-01-01 01:00:00,27.731651,54.225919,1411.064918,1,On,On,21.831384,Saturday,No,83.401855
2,2022-01-01 02:00:00,28.704277,58.907658,1755.715009,2,Off,Off,6.764672,Sunday,No,78.270888
3,2022-01-01 03:00:00,20.080469,50.371637,1452.316318,1,Off,On,8.623447,Wednesday,No,56.519850
4,2022-01-01 04:00:00,23.097359,51.401421,1094.130359,9,On,Off,3.071969,Friday,No,70.811732


In [26]:
time_col = None
for c in df.columns:
    if 'time' in c.lower() or 'date' in c.lower():
        time_col = c
        break

if time_col is None:
    time_col = 'Timestamp'

if time_col is None or time_col not in df.columns:
    raise ValueError("Could not find a timestamp column. Please ensure your file has 'time' or 'date' column.")

target_col = 'EnergyConsumption' if 'EnergyConsumption' in df.columns else df.columns[-1] # Corrected target column name

df[time_col] = pd.to_datetime(df[time_col], errors='coerce')
df = df.dropna(subset=[time_col])
df = df.set_index(time_col).sort_index()

numeric_df = df.select_dtypes(include=['number'])
if numeric_df.empty:
    raise ValueError("No numeric columns found! Please check dataset.")

df = numeric_df.resample('H').mean()   # hourly average for numeric data
df[target_col] = df[target_col].interpolate(method='time').ffill().bfill()

print("After resampling & cleaning:")
print(df.head())

After resampling & cleaning:
                     Temperature   Humidity  SquareFootage  Occupancy  \
Timestamp                                                               
2022-01-01 00:00:00    25.139433  43.431581    1565.693999        5.0   
2022-01-01 01:00:00    27.731651  54.225919    1411.064918        1.0   
2022-01-01 02:00:00    28.704277  58.907658    1755.715009        2.0   
2022-01-01 03:00:00    20.080469  50.371637    1452.316318        1.0   
2022-01-01 04:00:00    23.097359  51.401421    1094.130359        9.0   

                     RenewableEnergy  EnergyConsumption  
Timestamp                                                
2022-01-01 00:00:00         2.774699          75.364373  
2022-01-01 01:00:00        21.831384          83.401855  
2022-01-01 02:00:00         6.764672          78.270888  
2022-01-01 03:00:00         8.623447          56.519850  
2022-01-01 04:00:00         3.071969          70.811732  


/tmp/ipython-input-4095220225.py:23: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  df = numeric_df.resample('H').mean()   # hourly average for numeric data
